## DIA 037: Implementacion de Notificaciones Push en Tiempo Real para administradores

En este día, se añade una nueva capa de comunicación en tiempo real para la gestión de feedback. Cuando un usuario envía feedback acerca de las predicciones de la API, el sistema enviará automáticamente una notificación push a todos los administradores conectados mediante WebSockets. Esto se consigue integrando Flask-SocketIO en la aplicación. Los administradores, al conectarse, se unirán a un “room” especial denominado "admins", donde recibirán las notificaciones. De esta forma, se mejora la capacidad de respuesta y se agiliza la supervisión de la retroalimentación recibida.

Código Completo (api.py)
python
Copiar
from flask import Flask, Blueprint, request, jsonify, render_template, url_for
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_bcrypt import Bcrypt
from flask_mail import Mail, Message
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
from flask_socketio import SocketIO, emit, join_room
import os
import logging
from datetime import datetime
import time
import threading
import json

# Configuración básica y variables de entorno
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Inicialización de extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)
bcrypt = Bcrypt(app)
mail = Mail(app)
jwt = JWTManager(app)
limiter = Limiter(app, key_func=get_remote_address, default_limits=["200 per day", "50 per hour"])

# Inicialización de Flask-SocketIO
socketio = SocketIO(app, cors_allowed_origins="*")

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Modelos
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    # Otros campos omitidos para este ejemplo

class Feedback(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    prediction = db.Column(db.Integer, nullable=False)
    correct = db.Column(db.Boolean, nullable=False)
    comment = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "prediction": self.prediction,
            "correct": self.correct,
            "comment": self.comment,
            "timestamp": self.timestamp.isoformat()
        }

# ---------------------------
# Decorador para Roles (simplificado)
# ---------------------------
from functools import wraps
def role_required(required_role):
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            current_user_identity = get_jwt_identity()
            if not current_user_identity:
                return jsonify({"msg": "Token de acceso requerido"}), 401
            user = User.query.filter_by(username=current_user_identity).first()
            if not user:
                return jsonify({"msg": "Usuario no encontrado"}), 404
            # Se asume que el atributo 'role' existe; para este ejemplo, si no es "admin", se rechaza
            if getattr(user, 'role', 'user') != 'admin':
                return jsonify({"msg": "Acceso no autorizado"}), 403
            return f(*args, **kwargs)
        return wrapper
    return decorator

# ---------------------------
# Endpoints Comunes
# ---------------------------
@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    # Para este ejemplo, asumimos que el usuario existe y la validación es exitosa
    access_token = create_access_token(identity=username)
    logger.info(f"Usuario '{username}' inició sesión.")
    return jsonify(access_token=access_token), 200

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Endpoints de Versionado (v1 y v2)
# ---------------------------
api_v1 = Blueprint('api_v1', __name__)
@api_v1.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("100 per day")
def predict_v1():
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    # Simulación simple: retorna dígito 5 con 90% de probabilidad
    result = {"prediccion": 5, "probabilidad": 0.90, "version": "v1"}
    logger.info("v1: Predicción realizada.")
    return jsonify(result), 200

api_v2 = Blueprint('api_v2', __name__)
@api_v2.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("150 per day")
def predict_v2():
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    # Simulación mejorada: retorna dígito 7 con 95% de probabilidad y un mensaje adicional
    result = {"prediccion": 7, "probabilidad": 0.95, "version": "v2", "mensaje": "Predicción mejorada"}
    logger.info("v2: Predicción realizada con mejoras.")
    return jsonify(result), 200

app.register_blueprint(api_v1, url_prefix='/api/v1')
app.register_blueprint(api_v2, url_prefix='/api/v2')

# ---------------------------
# Endpoint de Feedback (ya implementado en días anteriores)
# ---------------------------
@app.route('/feedback', methods=['POST'])
@jwt_required()
def submit_feedback():
    current_user = get_jwt_identity()
    data = request.get_json()
    prediction = data.get('prediction')
    correct = data.get('correct')
    comment = data.get('comment', "")
    if prediction is None or correct is None:
        logger.warning(f"Feedback fallido: Datos incompletos del usuario '{current_user}'.")
        return jsonify({"msg": "Prediction and correct flag required"}), 400
    feedback = Feedback(
        username=current_user,
        prediction=prediction,
        correct=correct,
        comment=comment
    )
    db.session.add(feedback)
    db.session.commit()
    logger.info(f"Feedback registrado por '{current_user}': {feedback.to_dict()}")
    # Notificar a los administradores sobre el nuevo feedback
    socketio.emit('new_feedback', {"data": feedback.to_dict()}, room="admins")
    return jsonify({"msg": "Feedback submitted successfully"}), 201

# ---------------------------
# Endpoint para Reportes Automatizados
# ---------------------------
@app.route('/admin/report', methods=['GET'])
@jwt_required()
@role_required('admin')
def generate_report():
    total_feedback = Feedback.query.count()
    if total_feedback == 0:
        return jsonify({"msg": "No hay feedback disponible"}), 200
    correct_feedback = Feedback.query.filter_by(correct=True).count()
    incorrect_feedback = Feedback.query.filter_by(correct=False).count()
    accuracy = (correct_feedback / total_feedback) * 100
    report = {
        "total_feedback": total_feedback,
        "correct_feedback": correct_feedback,
        "incorrect_feedback": incorrect_feedback,
        "accuracy_percentage": accuracy
    }
    logger.info("Reporte generado: " + json.dumps(report))
    return jsonify(report), 200

# ---------------------------
# Endpoint de Retraining Basado en Feedback
# ---------------------------
@app.route('/admin/retrain', methods=['POST'])
@jwt_required()
@role_required('admin')
def retrain_model_endpoint():
    def retrain_job():
        logger.info("Inicio del retraining del modelo basado en feedback...")
        time.sleep(10)  # Simula el tiempo de retraining
        with open('updated_model.h5', 'w') as f:
            f.write("Modelo actualizado basado en feedback")
        logger.info("Retraining completado. Modelo actualizado.")
    thread = threading.Thread(target=retrain_job)
    thread.start()
    return jsonify({"msg": "Proceso de retraining iniciado"}), 202

# ---------------------------
# Implementación de Notificaciones Push para Administradores (WebSocket)
# ---------------------------
@socketio.on('join_admin')
def on_join_admin(data):
    # Se espera que los administradores envíen este evento para unirse al room "admins"
    join_room("admins")
    emit("message", {"data": "Te has unido a la sala de notificaciones de administradores."})

# ---------------------------
# Endpoint de Health Check
# ---------------------------
@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    socketio.run(app, debug=True)
Explicación del Código
Configuración y Extensiones:
La aplicación se configura con JWT, SQLAlchemy, Flask-Mail, Flask-Limiter y, ahora, Flask-SocketIO para notificaciones en tiempo real. Se establece el logging para registrar eventos importantes.

Versionado de la API:
Se definen dos Blueprints (api_v1 y api_v2) para el endpoint /predict, permitiendo la evolución de la API sin afectar a los clientes existentes.

Feedback y Reportes:
Se implementa el endpoint /feedback para que los usuarios envíen retroalimentación. Al registrarse feedback, se emite un evento new_feedback a los administradores conectados al room "admins". Además, el endpoint /admin/report genera un resumen del feedback.

Pipeline de Retraining:
El endpoint /admin/retrain inicia un proceso asíncrono de retraining (simulado) que actualiza el modelo, permitiendo la mejora continua basándose en los datos de feedback.

Notificaciones Push para Administradores:
Se añade un evento join_admin para que los administradores puedan unirse a la sala "admins". Así, cuando se recibe feedback, el servidor notifica en tiempo real a todos los administradores conectados mediante WebSocket.

Ejecución de la Aplicación:
La aplicación se ejecuta utilizando socketio.run(app, debug=True), lo que integra tanto HTTP como WebSocket.

Este ejemplo demuestra cómo integrar reportes automatizados, retraining y notificaciones push en tiempo real para una API de predicción, aprovechando la retroalimentación de los usuarios y permitiendo a los administradores estar al tanto de las novedades sin interrumpir el servicio.

